# Script to download wekeo data from HDA python library. It will create a folder in your current directory. After all rasters have been downloaded, it will mosaic the tiles by date and phenometric, and then will crop them with the shapefile boundaries, keeping just the *_mosaic_rec.tif files for every date/phenometrics combination

In [ ]:
import os
import fiona
import rasterio
import numpy as np
from hda import Client
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from fiona.transform import transform
from pyproj import Proj

c = Client(debug=True)


class wekeo_download:
    
    
    def __init__(self, dataset, shape, dates, products):
        
        if not os.path.exists('patata'):
            os.mkdir('patata')
            
        os.chdir('patata')
            
        self.dataset = dataset
        self.shape = shape
        self.dates = dates
        self.products = products
        #self.datasetlists = {'VPP': "EO:EEA:DAT:CLMS_HRVPP_VPP"}
        self.datasetlists = {'VPP_Index': "EO:EEA:DAT:CLMS_HRVPP_VI", 'VPP_Pheno': 'EO:EEA:DAT:CLMS_HRVPP_VPP'}
        self.variables = {'VPP_Index': ['PPI', 'NDVI', 'LAI', 'FAPAR'], 
                          'VPP_Pheno': ['SOSD', 'SOSV', 'MAXD', 'MAXV', 'EOSD', 'EOSV']}
        
        '''      
        PhenoMetrics
        ['MINV', 'MAXD', 'LENGTH', 'SOSD', 'QFLAG', 
        'EOSV', 'TPROD', 'MAXV', 'AMPL', 'SOSV', 'LSLOPE', 'EOSD', 'RSLOPE', 'SPROD']'''
               
        
        self.crs = fiona.open(shape).crs
        self.bbox = fiona.open(shape).bounds
              
        self.query_fix = {
              "datasetId": self.datasetlists[self.dataset],
              "boundingBoxValues": [
                {
                  "name": "bbox",
                  "bbox": [
                    transform(self.crs, 'EPSG:4326', [self.bbox[0]], [self.bbox[1]])[0][0], #a['Lon'].min(), #-6.614,
                    transform(self.crs, 'EPSG:4326', [self.bbox[0]], [self.bbox[1]])[1][0], #a['Lat'].min(), #36.945,
                    transform(self.crs, 'EPSG:4326', [self.bbox[2]], [self.bbox[3]])[0][0], #a['Lon'].max(), #-6.381,
                    transform(self.crs, 'EPSG:4326', [self.bbox[2]], [self.bbox[3]])[1][0] #a['Lat'].max(), #37.047
                  ]
                }
              ],
              "dateRangeSelectValues": [
                        {
                          "name": "temporal_interval",
                          "start": self.dates[0],
                          "end": self.dates[1]
                        }
              ]}
            
            
            
        self.query_moving = {"stringChoiceValues": [
                {
                  "name": "productType",
                  "value": self.products[0]
                },
                        {
                  "name": "productGroupId",
                  "value": "s1"
                }
              ]
            }
        
        self.query_moving_ = {"stringChoiceValues": [
                {
                  "name": "productType",
                  "value": self.products[0]
                }                        
              ]
            }
        
        #self.query = {**self.query_fix, **self.query_moving}
        
        
    def download(self):
        
        if self.dataset == 'VPP_Pheno':
            
            self.query = {**self.query_fix, **self.query_moving}
            
        else: 
            
            self.query = {**self.query_fix, **self.query_moving_}
            
            
        for p in self.products:

            try:
                print('Getting', p)

                self.query['stringChoiceValues'][0]['value'] = p
                matches = c.search(self.query)
                print(matches)
                matches.download()

            except Exception as e:
                print(e)
                continue 
                
        
        
    def mosaic(self):
        
        
        path = os.getcwd()
        
        rasters = {}
        for i in os.listdir(path): 
            if i.endswith('.tif'):

                year = i.split('_')[1][:8]       
                rasters[year] = {}
                
                
        for i in os.listdir(path): 
            if i.endswith('.tif'):
                year = i.split('_')[1][:8]
                phenometric = i.split('_')[-1].split('.')[0]
                rasters[year][phenometric]=[]

        for i in os.listdir(path): 
            if i.endswith('.tif'):
                year = i.split('_')[1][:8]
                phenometric = i.split('_')[-1].split('.')[0]
                tile = i.split('_')[3].split('-')[0]
                rasters[year][phenometric].append(os.path.join(path, i))
        
        
        for k, v in rasters.items():
            
            print(k,v)
    
            for i in v.keys():
            
                try:
                    # Mosaicking list, it turns empty after every loop
                    nrasters = []

                    rs = rasters[k][i]
                    print('RS', rs)
                    date = rs[0].split('_')[1][:8]
                    sat = rs[0].split('_')[2][:2]
                    tile = rs[0].split('_')[3].split('-')[0]
                    resolution = rs[0].split('_')[3].split('-')[1]
                    season = rs[0].split('_')[5]
                    phenometric = rs[0].split('_')[-1].split('.')[0]

                    out_mosaic = os.path.join(path, ('_').join(['mosaic', date, sat, resolution, season, phenometric, '.tif']))
                    out_mosaic_rec = os.path.join(path, ('_').join(['mosaic', date, sat, resolution, season, phenometric, 'rec.tif']))


                    for current in rs:
                        
                        src = rasterio.open(current)
                        nrasters.append(src)

                    mosaic, out_trans = merge(nrasters)

                    out_meta = src.meta.copy()

                    out_meta.update({"driver": "GTiff",
                        "height": mosaic.shape[1],
                        "width": mosaic.shape[2],
                        "transform": out_trans,
                        "crs": self.crs 
                    })


                    with rasterio.open(out_mosaic, "w", **out_meta) as dest:
                        dest.write(mosaic)

                    # Read Shape file
                    with fiona.open(self.shape, "r") as shp:
                        shapes = [feature["geometry"] for feature in shp]

                    # Read imagery file
                    with rasterio.open(out_mosaic) as src_:
                        out_image, out_transform = rasterio.mask.mask(src_, shapes, crop=True)
                        out_meta = src_.meta


                    # Save clipped imagery
                    out_meta.update({"driver": "GTiff",
                                     "height": out_image.shape[1],
                                     "width": out_image.shape[2],
                                     "transform": out_transform})


                    with rasterio.open(out_mosaic_rec, "w", **out_meta) as dest:
                        dest.write(out_image)
                        
                except Exception as e:
                    print(e)
                    continue
                    
    def clean(self):
        
        for i in os.listdir(os.getcwd()):
            if i.endswith('.tif') and not '_rec' in i:
                os.remove(os.path.join(os.getcwd(), i))
                
                
    def run(self):
                
        print('Downloading images...')
        self.download()
        print('Mosaicking and clipping...')
        self.mosaic()
        print('cleaning the folder...')
        self.clean()

### Note that in the class definition this parameters are required: dataset, shape, dates, products. To perform the crop the shapefile should be projected. If you want to keep the whole tiles just call the download method. It would be possible apply the mosaic and clean methods 

In [ ]:
a = wekeo_download('VPP_Pheno', '/media/diego/Datos3/EBD/eLTER/PhenoApp/shapes/Donana_32629.shp', ['2019-01-01', '2021-12-31'], ['EOSD'])

In [ ]:
a.run()